# Synchronize homework files

In [ ]:
! rm -rf sample_data; curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/sync_hw.py > sync_hw.py
files = ["quadrotor.py", "stability_analysis.py", "create_animation.py", "quad_sim.py"]

from sync_hw import sync_hw
sync_hw(6, files, True)

# Install Drake and set up a server for the visualizer

In [ ]:
# Install drake. We are using the lqr controller in drake as the final stabilizing controller.
# The installation process will take about 2 minutes but it's only required in the start of the Colab's virtual machine.
!curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/colab_drake_setup.py > colab_drake_setup.py
from colab_drake_setup import setup
setup()

# Problem 1 - Model Predictive Control

Creates the Quadrotor object and loads in the animation code.

**Re-run this block anytime you make change to `quadrotor.py`.**

In [ ]:
%matplotlib inline

"""
Simulate quadrotor
"""

import numpy as np
from math import sin, cos, pi
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import importlib

from quad_sim import simulate_quadrotor

# Need to reload the module to use the latest code
import quadrotor
importlib.reload(quadrotor)
from quadrotor import Quadrotor

"""
Load in the animation function
"""
import create_animation
importlib.reload(create_animation)
from create_animation import create_animation

# Weights of LQR cost
R = np.eye(2);
Q = np.diag([10, 10, 1, 1, 1, 1]);
Qf = Q;

# End time of the simulation
tf = 10;

# Construct our quadrotor controller 
quadrotor = Quadrotor(Q, R, Qf);

## Simulate the quadrotor using MPC as the controller


In [ ]:
# Set quadrotor's initial state and simulate
x0 = np.array([0.5, 0.5, 0, 1, 1, 0])
x, u, t = simulate_quadrotor(x0, tf, quadrotor)

anim, fig = create_animation(x, tf)
plt.close()
anim

## Simulate the quadrotor using LQR as the controller (Already implemented)

In [ ]:
# Set quadrotor's initial state and simulate
x0 = np.array([0.5, 0.5, 0, 1, 1, 0])
x, u, t = simulate_quadrotor(x0, tf, quadrotor, False)

anim, fig = create_animation(x, tf)
plt.close()
anim

# Problem 2 - Stability Analysis using Sums of Squares

In [ ]:
# Need to reload the module to use the latest code
import stability_analysis
importlib.reload(stability_analysis)
from stability_analysis import *

S_sol = stability_analysis.compute_region_of_attraction(quadrotor)
np.save('S_sol', S_sol)
stability_analysis.visualize_region_of_attraction(S_sol)

# Problem 3 - Control Lyapunov Functions

## MPC with CLF

In [ ]:
# Set quadrotor's initial state and simulate
x0 = np.array([0.5, 0.5, 0, 1, 1, 0])
x, u, t = simulate_quadrotor(x0, tf, quadrotor, True, True, False)

anim, fig = create_animation(x, tf)
plt.close()
anim

# QP with CLF

In [ ]:
# Set quadrotor's initial state and simulate using QP CLF as the controller
x0 = np.array([0.5, 0.5, 0, 0, 0, 0])
x, u, t = simulate_quadrotor(x0, tf, quadrotor, use_mpc=False, use_mpc_with_clf=False, use_clf_qp=True)

anim, fig = create_animation(x, tf)
plt.close()
anim